In [1]:
import os
import numpy as np
import cv2

In [2]:
y = np.load("train_labels.npy")
X = np.load("normalized_train_rgb_data.npy")

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2, l1_l2
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
output = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = output)

#Freeze first layers
for layer in base_model.layers[:20]:
    layer.trainable = False
    
#Unfreeze the remaining layers to allow fine-tuning
for layer in base_model.layers[20:]:
    layer.trainable = True

In [6]:
from keras.optimizers import Adam
from keras import metrics
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [7]:
history = model.fit(X, y, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
176/176 [==============================] - 59s 275ms/step - loss: 1.9650 - accuracy: 0.4005 - precision: 0.6542 - recall: 0.2188 - val_loss: 3.5974 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
176/176 [==============================] - 54s 309ms/step - loss: 1.0185 - accuracy: 0.6530 - precision: 0.7837 - recall: 0.5410 - val_loss: 2.7285 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
176/176 [==============================] - 55s 310ms/step - loss: 0.7195 - accuracy: 0.7462 - precision: 0.8313 - recall: 0.6712 - val_loss: 3.5082 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
176/176 [==============================] - 54s 309ms/step - loss: 0.4945 - accuracy: 0.8228 - precision: 0.8770 - recall: 0.7782 - val_loss: 4.3062 - val_accuracy: 6.0000e-04 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
176/176 [========================

In [8]:
model.save("WildSeekAI.keras")